In [12]:
import numpy as np
import cv2
from PIL import Image
import os
import pickle


In [13]:
base_dir = os.path.dirname(os.path.abspath('frec.ipynb'))
img_dir = os.path.join(base_dir,"images")
# print(img_dir)
face_cascade = cv2.CascadeClassifier("haar_cascade/haarcascade_frontalface_default.xml")
recognizer = cv2.face.LBPHFaceRecognizer_create()


label_id = {}
x_train = []
y_train = []
id=0

for root,dirs,files in  os.walk(img_dir):
#     print(root)
    for file in files:
        if file.endswith(".jpg") or file.endswith(".png"):
            path = os.path.join(root,file)
            name = os.path.basename(root)
            
            if not name in label_id:
                label_id[name]=id
                id+=1
            id_ = label_id[name]
            
            
            pil_img = Image.open(path).convert('L')#gray scale
            size = (550, 550)
            final_image = pil_img.resize(size, Image.ANTIALIAS)
            arr_img = np.array(final_image,'uint8')#uint8 default for open cv
            faces = face_cascade.detectMultiScale(arr_img,1.5,5)
            
            for (x,y,w,h) in faces:
                roi_img = arr_img[y:y+h,x:x+w]
                x_train.append(roi_img)
                y_train.append(id_)
                
                
with open("frec_labels.pickle","wb") as f: 
    pickle.dump(label_id,f)
# print(len(y_train),len(x_train))
recognizer.train(x_train,np.array(y_train))
recognizer.save("trainer.yml")
